# This Python script can be used to generate predictions for new molecules for the chance of RNA vs Protein binding. 

## Software License:

MIT License

Copyright (c) 2022 Yazdani et al. 

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

## Jupyter notebook within the Anaconda platform was used in writing the following script. Python version 3.8.2 was used as the coding language. 
$\;\;\;\;\;\;$



![Python logo](https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Python-logo-notext.svg/300px-Python-logo-notext.svg.png)

$\;\;\;\;\;\;$

In [ ]:
# Check Python version
from platform import python_version
print("Python version:")
print(python_version())

## macOS Catalina version 10.15.7 was used when running this code on the Anaconda platform. 

## This code was written with the following package versions. Please install the following packages with the mentioned versions for consistency of results.

### pandas --> version 1.4.1

In [ ]:
# Check pandas version
import pandas as pd
print("pandas version:")
print(pd.__version__)

### numpy --> version 1.20.3

In [ ]:
# Check numpy version
import numpy as np
print("numpy version:")
print(np.__version__)

### tensorflow --> version 2.3.1

In [ ]:
# Check tensorflow version
import tensorflow as tf
print("tensorflow version:")
print(tf.__version__)

### keras --> version 2.4.0


In [ ]:
# Check keras version
print("keras version:")
print(tf.keras.__version__)

In [ ]:
# Remove warnings
import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
# Import the required packages for data processing and running the MLP model
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import pickle

# Gets current directory of the jupyter notebook to use for directories of different files
folder_dir = os.getcwd()[:-10]

# Load medians of features from the training set to fill in missing values of the new compound set
df_medians = pd.read_csv(folder_dir + "/ML_CSVs/df_all_3_feature_medians.csv")

# Get features assined to medians as a dictionary
median_values_dict = dict(zip(df_medians.feature, df_medians.median_value))

# Get the features of the training set for use later
features = df_medians["feature"].tolist()

# Load in the saved Keras model
reconstructed_model = keras.models.load_model(folder_dir + "/MLP_saved_model")

print("Processes complete!")

In [ ]:
# Preparation of the new compounds array to be run in the MLP model

# Read in Mordred features of the new compound set
# As an example, the input here is the 8 compounds shown as test cases in the paper. 
df_Mordred_extra = pd.read_csv(folder_dir + "/data/Mordred_files/Mordred_Test_Compounds_3D.csv")

# Take the compound names as a list as they will be used later
Mordred_extra_names = df_Mordred_extra["name"].tolist()

# Remove the name column
df_Mordred_extra_no_name = df_Mordred_extra.iloc[:, 1:]

# Only keep features of the new compounds that also exist in df_all_3
for item in df_Mordred_extra_no_name:
    if item not in features:
        df_Mordred_extra_no_name = df_Mordred_extra_no_name.drop(item, axis=1)

# Replace infinity in the feature values by nan values
df_Mordred_extra_no_name.replace([np.inf, -np.inf], np.nan, inplace=True)     

# Fill nan values of the test matrix with median of df_all_3 columns
for i in range(8):
    for item in features:
        if np.isnan(df_Mordred_extra_no_name.at[i, item]) == True:
            df_Mordred_extra_no_name.at[i, item] = median_values_dict[item]
            
df_Mordred_extra_no_name

In [ ]:
# Run the model on the new compound dataset

# Load in the standardscalar package
from sklearn.preprocessing import StandardScaler

# Load in the standardization used in standardizing the training set
sc = pickle.load(open(folder_dir + "/ML_CSVs/scaler.pkl", 'rb'))

# Standardize the new compound dataset using the standardization parameters derived from the training set
X_extra_test = df_Mordred_extra_no_name.values
X_extra_test = sc.transform(X_extra_test)

# Use the MLP model to predict the chance of RNA binding over protein binding
prediction = [item[0] for item in reconstructed_model.predict(X_extra_test).tolist()]

# Print compound name followed by the probability predicted by model
for i in range(len(prediction)):
    print(Mordred_extra_names[i])
    print(round(prediction[i]*100, 1))
    print()